<a href="https://colab.research.google.com/github/machiwao/CCTHESS1-CCTHESS2-Dev-and-Docs/blob/jessy/Albedo_data_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import ee
import geemap
import pandas as pd
from scipy.signal import savgol_filter

In [15]:
# Authenticate and initialize Earth Engine
cloud_project = 'heat-index-forecasting'

try:
  ee.Initialize(project=cloud_project)
except:
  ee.Authenticate()
  ee.Initialize(project=cloud_project)

In [16]:
stations = {
    "Sinait": ee.Geometry.Point([120.459762, 17.89019]).buffer(25000),
    "Tayabas": ee.Geometry.Point([121.596575, 14.018428]).buffer(25000),
    "Tanay": ee.Geometry.Point([121.36927, 14.581167]).buffer(25000),
    "Tuguegarao": ee.Geometry.Point([121.758469, 17.647678]).buffer(25000),
    "Virac": ee.Geometry.Point([124.209834, 13.576558]).buffer(25000),
}

start_date = "2014-01-01"
end_date   = "2024-01-01"

In [18]:
def extract_albedo(geometry, station_name, start_date, end_date):

    albedo = ee.ImageCollection("LANDSAT/LC08/C02/T2_L2").filterDate(start_date, end_date).filterBounds(geometry)

    print(f"\n--- Processing {station_name} ---")

    def process(img):
        # Landsat 8 Collection 2 Level-2 SR bands are scaled integers.
        # According to the USGS Landsat Collection 2 Science Product Guide:
        # Reflectance = DN * 0.0000275 - 0.2
        # Reference: https://www.usgs.gov/landsat-missions/landsat-collection-2-level-2-science-products
        scale = 0.0000275
        offset = -0.2

        # Apply scale and offset to surface reflectance bands
        b2 = img.select("SR_B2").multiply(scale).add(offset)  # Blue
        b4 = img.select("SR_B4").multiply(scale).add(offset)  # Red
        b5 = img.select("SR_B5").multiply(scale).add(offset)  # NIR
        b6 = img.select("SR_B6").multiply(scale).add(offset)  # SWIR1
        b7 = img.select("SR_B7").multiply(scale).add(offset)  # SWIR2

        # Albedo equation from literature (coefficients from Liang 2001)
        albedo_val = (
            b2.multiply(0.356)
            .add(b4.multiply(0.130))
            .add(b5.multiply(0.373))
            .add(b6.multiply(0.085))
            .add(b7.multiply(0.072))
            .subtract(0.0018)
        ).divide(1.016).rename("Albedo")

        return ee.Feature(
            None,
            {
                "date": img.date().format("YYYY-MM-dd"),
                "Albedo": albedo_val.reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=geometry,
                    bestEffort=True,
                    maxPixels=1e9
                ).get("Albedo"),
            },
        )

    features = albedo.map(process)
    features = ee.FeatureCollection(features)

    # Convert to DataFrame
    df = geemap.ee_to_df(features)

    if df.empty:
        print(f"No data for {station_name}")
        return None

    df["date"] = pd.to_datetime(df["date"])
    df = df.sort_values("date")

    # Handle duplicate dates by grouping and averaging
    df = df.groupby("date").mean().reset_index()

    # Define the strict date range
    full_range = pd.date_range(start=start_date, end=end_date, freq="D")

    # Reindex to the strict full date range
    df = df.set_index("date").reindex(full_range)

    # Interpolate missing values
    df["Albedo"] = pd.to_numeric(df["Albedo"], errors="coerce")
    df["Albedo"] = df["Albedo"].interpolate(method="linear").ffill().bfill()

    df.index.name = "date"
    df = df.reset_index().rename(columns={"index": "date"})
    df["station"] = station_name

    # Save to CSV
    filename = f"{station_name}_Albedo.csv"
    df.to_csv(filename, index=False)
    print(f"Saved {filename} with {df.shape[0]} rows")

    return df

In [19]:
all_dfs = {}
for station, geom in stations.items():
    df_station = extract_albedo(geom, station, start_date, end_date)
    if df_station is not None:
        all_dfs[station] = df_station

print("\nProcessing complete!")


--- Processing Sinait ---
Saved Sinait_Albedo.csv with 3653 rows

--- Processing Tayabas ---
Saved Tayabas_Albedo.csv with 3653 rows

--- Processing Tanay ---
Saved Tanay_Albedo.csv with 3653 rows

--- Processing Tuguegarao ---
Saved Tuguegarao_Albedo.csv with 3653 rows

--- Processing Virac ---
Saved Virac_Albedo.csv with 3653 rows

Processing complete!


In [20]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import glob

# # Load all station CSVs
# csv_files = glob.glob("*_Albedo.csv")

# plt.figure(figsize=(14, 6))

# for file in csv_files:
#     df = pd.read_csv(file, parse_dates=["date"])
#     station_name = file.split("_")[0].capitalize()

#     # Plot BHA
#     plt.plot(df["date"], df["BHA"], label=station_name)

# plt.title("Daily BHA Values (Interpolated) for All Stations")
# plt.xlabel("Date")
# plt.ylabel("BHA (scaled reflectance)")
# plt.legend()
# plt.grid(True, linestyle="--", alpha=0.6)
# plt.tight_layout()
# plt.show()
